In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Los datos corresponden a un subset del dataset cats vs dogs de https://www.kaggle.com/c/dogs-vs-cats
files=glob.glob('/content/drive/MyDrive/Machine_Learning/CATS_DOGS/*.jpg')

In [ ]:
print(files[:10])

['/content/drive/MyDrive/Machine_Learning/CATS_DOGS/cat.9249.jpg', '/content/drive/MyDrive/Machine_Learning/CATS_DOGS/cat.8745.jpg', '/content/drive/MyDrive/Machine_Learning/CATS_DOGS/cat.9261.jpg', '/content/drive/MyDrive/Machine_Learning/CATS_DOGS/cat.8787.jpg', '/content/drive/MyDrive/Machine_Learning/CATS_DOGS/cat.9467.jpg', '/content/drive/MyDrive/Machine_Learning/CATS_DOGS/cat.9315.jpg', '/content/drive/MyDrive/Machine_Learning/CATS_DOGS/cat.8802.jpg', '/content/drive/MyDrive/Machine_Learning/CATS_DOGS/cat.9458.jpg', '/content/drive/MyDrive/Machine_Learning/CATS_DOGS/cat.8630.jpg', '/content/drive/MyDrive/Machine_Learning/CATS_DOGS/cat.8751.jpg']


**Training dataset**

In [ ]:
size = 64

train_data = np.array([
    np.asarray(Image.open(file).resize((size, size)))
    for file in files[:200]
])

train_label = np.array([
    1 if 'dog' in file else 0
    for file in files[:200]
])

test_label = np.array([
    1 if 'dog' in file else 0
    for file in files[200:]
])

test_data = np.array([
    np.asarray(Image.open(file).resize((size, size)))
    for file in files[200:]
])

In [ ]:
train_data_flatten = train_data.reshape(train_data.shape[0], -1).T
train_set_label=train_label.reshape((1,train_label.shape[0]))
test_set_label=test_label.reshape((1,test_label.shape[0]))
test_data_flatten = test_data.reshape(test_data.shape[0], -1).T

In [ ]:
test_data_flatten.shape

(12288, 1617)

In [ ]:
np.max(train_data_flatten)

np.uint8(255)

**Normalization**

In [ ]:
train_set_data= train_data_flatten/255
test_set_data = test_data_flatten/np.max(test_data_flatten)

In [ ]:
train_set_data.shape, test_set_data.shape

((12288, 200), (12288, 1617))

**Sigmoid function:**

$a = \sigma(z)=\frac{1}{1+e^{-z}}$

where

$z = \omega_{0} x + \omega_{1}$

In [ ]:
def sigmoid(z):
  a = 1/(1+np.exp(-z))
  return a

In [ ]:
w,b = np.zeros((len(train_set_data),1)),0
l = sigmoid(np.matmul(w.T,train_set_data)+b)
l.shape

(1, 200)

**Cost function:**

$J(w,b)= -\frac{1}{m}\sum{[y^{(i)}\log{\hat{y}^{(i)}}+(1-y^{(i)})\log{(1-\hat{y}^{(i)}})]}$

$\omega =  \omega -\frac{1}{m}\alpha [X\cdot(a-y)]$

$b = b - \frac{1}{m}\alpha(a-y)$

In [ ]:
def grad(x,y,w,b, iteraciones, alpha):
  m = x.shape[1]
  w,b = np.zeros((len(train_set_data),1)),0 #Inicializamos los pesos en cero
  for i in range(iteraciones):
    suma = 0 #Defino la variable donde guardaré el valor de la sumatoria para el cálculo del costo
    a = sigmoid(np.matmul(w.T,x)+b) #Calcula la función de activación con w,b iguales a cero en principio
    w = w-alpha*(1/m)*np.matmul(x,(a-y).T) #Actualizamos w usando dw
    db = np.mean(a - y)
    b = b - alpha * db #Actualizamos b usando db
    #b = b-alpha*(1/m)*(a-y) #Actualizamos b usando db
    for j in range(a.shape[1]): #Hacemos el cálculo del costo con el w,b iniciales
      suma += y[:1,j]*np.log(a[:1,j])+(1-y[:1,j])*np.log(1-(a[:1,j]))
    J = -(1/m)*sum(suma) #Calculamos el costo
    f = i%100
    if f == 0:
      print('J en la ',i,'-ésima iteracion:', J) #Imprimimos el costo en cada iteración
  return w, b, J

In [ ]:
w, b, J = grad(x=train_set_data, y=train_set_label, w=w,b=b, iteraciones=1000, alpha=0.01) #Realizo el grad. desc. con el conjunto de entrenamiento

J en la  0 -ésima iteracion: 0.6931471805599465
J en la  100 -ésima iteracion: 3.7545638575209317
J en la  200 -ésima iteracion: 0.35837671027775114
J en la  300 -ésima iteracion: 0.3548329944029582
J en la  400 -ésima iteracion: 0.5779058286679104
J en la  500 -ésima iteracion: 0.6002544608715724
J en la  600 -ésima iteracion: 0.4734175801340802
J en la  700 -ésima iteracion: 0.13124266654501412
J en la  800 -ésima iteracion: 0.10720760336046664
J en la  900 -ésima iteracion: 0.09672796037480971


In [ ]:
def binary(x, y, w, b ,imagen):
  p = sigmoid(np.matmul(w.T, x)+b)
  ytst = np.matmul(w.T, x)+b
  ytst = -(ytst/100) #Usamos la función de activación con los pesos finales
  if ytst[:1,imagen]< 0.5:
    ytst[:1,imagen]=0 #Evaluamos en la imagen específica
    print('La imagen ',imagen,' es un perro')
  if ytst[:1,imagen] > 0.5:
    ytst[:1,imagen]=1
    print('La imagen ',imagen,' es un gato')
  return ytst

In [ ]:
twst2 = binary(train_set_data, train_set_label, r[0], r[1], 75) #Calcula todas las predicciones del dataset, pero solo imprime la que se pasa en el último arg.

La imagen  75  es un perro


In [ ]:
twst2 = binary(train_set_data, train_set_label, r[0], r[1], 79)

La imagen  79  es un perro


In [ ]:
#plt.imshow(train_data[79])

In [ ]:
testing_precission = binary(test_set_data, test_label, w, b, 1) #Aunque solo imprime la imagen seleccionada en el último argumento de la función
                                                                #Realmente está calculando la predicción en todas las imágenes del dataset

La imagen  1  es un perro


In [ ]:
def precission(data, labels):
  y_test = np.zeros_like(data[0])
  for i in range(len(testing_precission[0].shape)):
    if testing_precission[0, i] > 0.5:
      y_test[i]=1
    else:
      y_test[i]=0

  from sklearn.metrics import accuracy_score

  accuracy = accuracy_score(y_test, labels)
  print('La precisión es del ',accuracy,'%')

  return accuracy

In [ ]:
precission = precission(testing_precission, test_label)

La precisión es del  0.505256648113791 %
